In [3]:
import os
os.chdir('..')


In [9]:
from tonal import pianoroll2number, joint_prob2pianoroll96
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from pypianoroll import Multitrack, Track
import pypianoroll as pr
import pickle
from matplotlib import pyplot as plt
import os
import random
from model.VAE import *

In [1]:
pwd

'/mnt/md1/user_victor/automatic_melody_harmonization/notebook'

In [13]:
# device = torch.device('cuda:1')
device = 'cpu'
val_size = 500

# Load data
print('loading data...')
# melody_data = np.load('./melody_data.npy')
# chord_groundtruth = np.load('./chord_groundtruth.npy')
chord_onehot = np.load('./onehot_96.npy')

# melody = np.load('./melody_baseline.npy')
lengths = np.load('./length.npy')

f = open('tempos', 'rb')
tempos = pickle.load(f)
f.close()
f = open('downbeats', 'rb')
downbeats = pickle.load(f)
f.close()

print('splitting testing set...')
# melody_data = melody_data[:val_size]
# chord_groundtruth = chord_groundtruth[:val_size]

# val_melody = torch.from_numpy(melody[:val_size]).float()
val_chord = torch.from_numpy(chord_onehot[:val_size]).float()
val_length = torch.from_numpy(lengths[:val_size])

tempos = tempos[:val_size]
downbeats = downbeats[:val_size]


loading data...
splitting testing set...


In [14]:

# Load model
print('building model...')
model = VAE(device = device).to(device)
model.load_state_dict(torch.load('output_models/model_vae.pth'))

model.eval()
chord, length = val_chord.to(device), val_length.to(device)

# Prediction
chord_pred, _, mu, log_var, input_x = model(chord,length)

building model...


In [21]:
chord_pred[0]

array([[2.5184202e-06, 1.5244591e-17, 6.2841087e-12, ..., 1.9881743e-10,
        5.2519209e-16, 2.7624528e-12],
       [1.1105512e-06, 2.3133952e-25, 1.0876592e-18, ..., 6.2585478e-17,
        1.2020495e-22, 9.6774623e-18],
       [9.4075078e-01, 2.2639604e-06, 8.4427300e-13, ..., 5.3348527e-11,
        2.5747914e-11, 5.7785014e-12],
       ...,
       [4.2970033e-04, 4.5616929e-14, 1.1828368e-16, ..., 1.6002414e-15,
        5.4371147e-17, 1.0034591e-15],
       [1.2148623e-04, 5.7152498e-01, 2.0806915e-06, ..., 3.8566970e-05,
        1.2229836e-07, 3.8555108e-08],
       [1.0333993e-05, 9.7323483e-01, 7.9513729e-09, ..., 1.4840099e-06,
        2.7465873e-11, 3.0970768e-12]], dtype=float32)

In [16]:
chord.shape

torch.Size([500, 272, 96])

In [17]:
chord[0]

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [19]:
# Proceed chord decode
print('proceed chord decode...')
chord_pred = chord_pred.cpu().detach().numpy()
# melody = melody.reshape(val_size, 272, 48, 12)
# melody = melody.cpu().detach().numpy()
length = length.cpu().detach().numpy()
joint_prob = chord_pred

# Append argmax index to get pianoroll array
chord_pianoroll = []
for song in joint_prob:
    pianoroll = []
    for beat in song:
        pianoroll.append(joint_prob2pianoroll96(beat))
    chord_pianoroll.append(pianoroll)

chord_pianoroll = np.asarray(chord_pianoroll)

accompany_pianoroll = chord_pianoroll * 100
print(chord_pianoroll.shape)

proceed chord decode...
(500, 272, 128)


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])